This is used to download FAANG's data

In [ ]:
import yfinance as yf
import pandas as pd
import time

# List of company tickers you want to download options for
tickers = ["Meta", "AAPL", "AMZN", "NFLX", "GOOG"]  # Add more if needed

# Loop through each ticker
for ticker in tickers:
    print(f"Downloading option data for {ticker}...")
    stock = yf.Ticker(ticker)
    
    try:
        expiration_dates = stock.options
    except Exception as e:
        print(f"Could not fetch expiration dates for {ticker}: {e}")
        continue

    all_options = []

    for date in expiration_dates:
        try:
            chain = stock.option_chain(date)
            calls = chain.calls.copy()
            calls["optionType"] = "call"
            calls["expirationDate"] = date

            puts = chain.puts.copy()
            puts["optionType"] = "put"
            puts["expirationDate"] = date

            combined = pd.concat([calls, puts], ignore_index=True)
            combined["ticker"] = ticker  # Add ticker column
            all_options.append(combined)

            time.sleep(0.5)  # Respectful delay between requests

        except Exception as e:
            print(f"Failed to download options for {ticker} on {date}: {e}")
            continue

    if all_options:
        df_all = pd.concat(all_options, ignore_index=True)
        filename = f"{ticker}_all_options.csv"
        df_all.to_csv(filename, index=False)
        print(f"Saved {filename} with {len(df_all)} rows.\n")
    else:
        print(f"No option data saved for {ticker}.\n")

    